<a href="https://colab.research.google.com/github/auberr/sparta_hp_ai/blob/main/sparta_AI_4_Text_classification_prac_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습: Zero-shot Classification

이번 실습에서는 open LLM을 가지고 zero-shot classification을 해봅니다. 먼저 필요한 library들을 설치합시다.

In [ ]:
!pip install datasets transformers evaluate --quiet

import os
import random
import numpy as np
import evaluate
from datasets import load_dataset, DatasetDict
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

# 1) MNLI 데이터셋 로드: "nyu-mll/glue", "mnli"
#    split은 train, validation_matched, validation_mismatched 등 여러 개가 있을 수 있으나
#    요구사항: 학습 시 'train' split만 활용 (valid 또한 train에서 만들어야 함)
raw_datasets = load_dataset("nyu-mll/glue", "mnli")

# 2) 라벨 종류: MNLI는 3개 레이블(0: entailment, 1: neutral, 2: contradiction)
#    (label 인덱스는 실제 데이터 구조 확인 가능)
print(raw_datasets)

# 3) 모델 & 토크나이저 선택 (예: DistilBERT)
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

# 3) train split에서 10,000개만 사용
train_split_full = raw_datasets["train"]
# train split 총 데이터 길이
print("Original train dataset length:", len(train_split_full))

# 10,000개
max_train = 10000
if len(train_split_full) < max_train:
    max_train = len(train_split_full)
train_split_10k = train_split_full.select(range(max_train))

print("Reduced train dataset length:", len(train_split_10k))

# 4) 10,000개를 90%(학습) / 10%(검증) 분할
train_size = int(len(train_split_10k) * 0.9)
indices = list(range(len(train_split_10k)))
random.shuffle(indices)
train_indices = indices[:train_size]
valid_indices = indices[train_size:]

train_dataset = train_split_10k.select(train_indices)
valid_from_train_dataset = train_split_10k.select(valid_indices)

print(f"Train dataset size: {len(train_dataset)}, Valid from train size: {len(valid_from_train_dataset)}")


# 5) Tokenize 함수: premise/hypothesis 2문장을 하나로
def tokenize_function(examples):
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True
    )

# 6) 데이터셋 토큰화
train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_from_train_dataset = valid_from_train_dataset.map(tokenize_function, batched=True)

# 최종 평가 용도의 validation_matched
val_matched_dataset = raw_datasets["validation_matched"].map(tokenize_function, batched=True)

# 7) Padding Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 8) 평가 지표로 Accuracy 사용
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=preds, references=labels)

# 9) TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="mnli-distilbert-output",
    evaluation_strategy="epoch",   # 매 epoch마다 검증
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,            # 예시로 2 epoch
    weight_decay=0.01,
    logging_steps=50,
    logging_dir="logs",
    load_best_model_at_end=True,
    # 아래 2개 옵션으로 W&B 대신 Hugging Face Transformers만 사용
    report_to=["none"],            # wandb에 로그 X
    run_name="mnli-distilbert-run" # run_name 지정 (wandb 사용 안 함)
)

# 10) Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_from_train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 11) 모델 학습 (Colab 셀에 로그 남음)
train_result = trainer.train()
trainer.save_model()

# 12) "validation_matched" 성능 측정(학습/검증X, 최종 평가)
val_result = trainer.evaluate(eval_dataset=val_matched_dataset)
print("Validation matched result:", val_result)
print(f"Accuracy on validation_matched = {val_result['eval_accuracy']:.4f}")


DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Original train dataset length: 392702
Reduced train dataset length: 10000
Train dataset size: 9000, Valid from train size: 1000


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-1dc7c7c94e91>:105: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.870900,0.748942,0.677000
2,0.638100,0.710463,0.695000
3,0.464100,0.771421,0.711000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.870900,0.748942,0.677000
2,0.638100,0.710463,0.695000
3,0.464100,0.771421,0.711000
4,0.273000,0.967590,0.701000
5,0.196000,1.174536,0.705000
6,0.176900,1.424419,0.699000
7,0.071700,1.654369,0.707000
8,0.069600,1.778924,0.702000
9,0.021800,1.889823,0.692000
10,0.011700,1.903690,0.700000


Validation matched result: {'eval_loss': 0.7893429398536682, 'eval_accuracy': 0.6649006622516557, 'eval_runtime': 22.7024, 'eval_samples_per_second': 432.333, 'eval_steps_per_second': 27.046, 'epoch': 10.0}
Accuracy on validation_matched = 0.6649
축하합니다! 50% 이상 달성하였습니다.


In [ ]:
!pip install datasets

그 다음 Gemma-2B를 사용하기 위해 다음과 같은 작업을 진행합니다:
1. huggingface.co 계정 만들고 로그인하기
2. https://www.kaggle.com/models/google/gemma/license/consent 에서 Gemma license 동의하기
3. 홈 화면으로 돌아와, `Profile > Settings > Access Tokens` 메뉴로 들어와 "Write" type의 token 생성하기
4. 생성한 토큰을 아래 "HF TOKEN"에 불여넣고 셀을 실행하기.

In [ ]:
from huggingface_hub import login
from google.colab import userdata


login("")

정상적으로 token을 생성하고 Gemma license에 동의했다면 아래 코드로 tokenizer와 Gemma-2B 모델을 불러올 수 있습니다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

이번에는 Gemma-2B를 가지고 간단한 text 생성을 해봅시다.
"What is your name?" 이라는 text를 넣었을 때 어떤 text가 생성되는지 살펴봅시다.

In [ ]:
input_text = "What is your name?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

<bos>What is your name?

What is your age?

What is your gender?

What is your race?

What


2B의 작은 LLM이라 질좋은 답변이 나오지 않는 것을 알 수 있습니다.
이번에는 입력으로 넣어준 token들의 logit을 계산해봅시다.

In [ ]:
tokens = input_ids['input_ids']
print(tokens)

logits = model(**input_ids).logits
for i in range(tokens.shape[-1]):
    token = tokens[0, i].item()
    print(logits[0, i, token])

tensor([[     2,   1841,    603,    861,   1503, 235336]], device='cuda:0')
tensor(-18.2747, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-33.2665, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-23.9536, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-27.7627, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-19.6064, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-21.0372, device='cuda:0', grad_fn=<SelectBackward0>)


위와 같이 모델 출력의 `.logits`을 통해 token들의 logit을 알 수 있습니다.
Logit은 높을 수록 token이 나올 확률이 높다는 뜻입니다.

이번에는 logit 계산을 통해 zero-shot classification을 구현해보도록 하겠습니다.

In [ ]:
import torch

def zero_shot_classification(text, task_description, labels):  # text는 주어진 입력, task_description은 task에 대한 설명, labels은 class들을 text로 변환한 결과입니다.
    text_ids = tokenizer(task_description + text, return_tensors="pt").to("cuda")  # 먼저 task_description과 text를 이어붙인 후, tokenize합니다.
    probs = []
    for label in labels:  # 그 다음 각 text화된 label들을 tokenize하고 입력에 이어붙인 후, Gemma-2B에 넣어줍니다.
        label_ids = tokenizer(label, return_tensors="pt").to("cuda")
        n_label_tokens = label_ids['input_ids'].shape[-1] - 1  # text로 변환한 label의 token 수를 계산합니다.
        input_ids = {
            'input_ids': torch.concatenate([text_ids['input_ids'], label_ids['input_ids'][:, 1:]], axis=-1),  # concatenate 명령어를 통해 이어붙이는 모습입니다.
            'attention_mask': torch.concatenate([text_ids['attention_mask'], label_ids['attention_mask'][:, 1:]], axis=-1)
        }

        logits = model(**input_ids).logits  # Logit을 계산한 모습입니다.
        prob = 0
        n_total = input_ids['input_ids'].shape[-1]
        for i in range(n_label_tokens, 0, -1):  # 일반적으로 text로 변환한 label은 여러 token으로 이루어져있습니다. 이러한 label에 대한 logit은 구성하는 모든 token들의 logit들의 합으로 정의합니다.
            token = label_ids['input_ids'][0, i].item()
            prob += logits[0, n_total - i, token].item()
        probs.append(prob)

        del input_ids
        del logits
        torch.cuda.empty_cache()  # 위의 del과 empty_cache() 명령어를 통해 GPU를 제때 할당해제 해줍니다. 만약 GPU가 여유롭다면 지워주시는게 속도적으로 이득입니다.

    return probs

아래는 실제로 zero-shot classification을 해본 결과입니다.

In [ ]:
probs = zero_shot_classification("I am happy!", "Is the sentence positive or negative?: ", ["positive", "negative"])
print(probs)

[-4.5151824951171875, -9.59005069732666]


보시다시피 우리는 Gemma를 별도로 학습하지 않았음에도 불구하고 주어진 문장이 긍정적이라는 것을 정확하게 예측하고 있습니다.

다음은 영화 리뷰 감정 분석 task에 적용해봅시다.
먼저 data를 불러옵니다.

In [ ]:
from datasets import load_dataset


imdb = load_dataset("imdb")
def preprocess_function(examples):
    return tokenizer(examples["text"], max_length=200, truncation=True)

tokenized_imdb = imdb.map(preprocess_function, batched=True)

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

그리고 `test` data에서 50개의 영화 리뷰에 대해 예측하는 코드는 다음과 같습니다.

In [ ]:
import numpy as np
from tqdm import tqdm


n_corrects = 0
for i in tqdm(range(50)):
    text = tokenized_imdb['test'][i]['text']
    label = tokenized_imdb['test'][i]['label']
    probs = zero_shot_classification(
        text,
        "A movie review is given. Decide that the movie review is positive or negative: ",
        labels=["Answer: negative.", "Answer: positive."]
    )

    pred = np.argmax(np.array(probs))
    if pred == label:
        n_corrects += 1

print(n_corrects)

100%|██████████| 50/50 [00:49<00:00,  1.01it/s]

44


보시다시피 정확도 88%로, 매우 높은 성능을 보이는 것을 알 수 있습니다.